In [1]:
import h5py
import numpy as np
#!/usr/bin/env python
# -*- coding: utf-8 -*-

In [2]:
# Load in GA h5
work_dir ="C:\\Users\\DominiqueFournier\\Google Drive\\Research\\Tli_Kwi_Cho\\Modelling\\"
fileName ="Consortium_Oct2017.geoh5"

f = h5py.File(work_dir+fileName, 'r')

In [31]:
'hello'.encode('utf8')

b'hello'

In [11]:

keys = list(f['GEOSCIENCE']['Groups'].keys())
for key in keys:
    print(key.decode('utf8'))

AttributeError: 'str' object has no attribute 'decode'

In [54]:
aa=f['GEOSCIENCE']['Groups']['{079e2f2d-a850-43df-ab9b-b9c2c3e62146}']['Objects']['{37b32a94-c906-4c27-b940-549fb7a9850c}']['Data']

In [55]:
aa.name

'/GEOSCIENCE/Groups/{079e2f2d-a850-43df-ab9b-b9c2c3e62146}/Objects/{37b32a94-c906-4c27-b940-549fb7a9850c}/Data'

In [12]:
f

<HDF5 file "Consortium_Oct2017.geoh5" (mode r)>